# Programming 3: Lecture 2
## Crossing the wires:
## How to spread your computation over more than one computer

In [1]:
import multiprocessing as mp
from multiprocessing.managers import BaseManager, SyncManager
import os, sys, time, queue

# Distributed Computing over the Network
## If you can't get enough cores in one computer, get more computers!
* Most serious distributed computing problems can't be solved using the processors in only one computer
* Computer clusters of homogeneous resources (ie. computers of roughly the same type) are used in practice
* In addition to the sheer _number_ of compute resources, _reliability_ is also a key concern

# Network concepts
* You can communicate over the network using TCP/IP "packets": items of data of fixed length
* The TCP/IP protocol guarantees that the packets are routed to the correct destination
* It also makes sure that large datasets are split into multiple packets, and they are guaranteed to arrive, and reassembled
* It presents itself to the programmer as two "sockets" (unique IP address and PORT combination) and acts like it is a file ("IO Stream")
* You can use this abstraction directly using the Python "socket" module and read and write data between Python programs on different computers
* We will be going one step higher in the abstraction hierarchy though

## Defining a Manager
* The Manager is a "proxy object"; it offers access to shared resources under one name
* Confusingly, given this primary function, it is also what you use to communicate over the network!
* It can "listen" and "connect" on a TCP/IP socket; a unique IP and PORT combination
* You need to define it twice: on each "end" of a connection (or "in the middle" if you are using a one-to-many approach as we do here")
* This may be a good point to explain different distributed computing "cluster topologies"...

# Back to that manager thingie...

In [2]:
def make_server_manager(port, authkey):
    """ Create a manager for the server, listening on the given port.
        Return a manager object with get_job_q and get_result_q methods.
    """
    job_q = queue.Queue()
    result_q = queue.Queue()

    # This is based on the examples in the official docs of multiprocessing.
    # get_{job|result}_q return synchronized proxies for the actual Queue
    # objects.
    class QueueManager(BaseManager):
        pass

    QueueManager.register('get_job_q', callable=lambda: job_q)
    QueueManager.register('get_result_q', callable=lambda: result_q)

    manager = QueueManager(address=('', port), authkey=authkey)
    manager.start()
    print('Server started at port %s' % port)
    return manager

In [3]:
def runserver(fn, data):
    # Start a shared manager server and access its queues
    manager = make_server_manager(PORTNUM, b'whathasitgotinitspocketsesss?')
    shared_job_q = manager.get_job_q()
    shared_result_q = manager.get_result_q()
    
    if not data:
        print("Gimme something to do here!")
        return
    
    print("Sending data!")
    for d in data:
        shared_job_q.put({'fn' : fn, 'arg' : d})
    
    time.sleep(2)  
    
    results = []
    while True:
        try:
            result = shared_result_q.get_nowait()
            results.append(result)
            print("Got result!", result)
            if len(results) == len(data):
                print("Got all results!")
                break
        except queue.Empty:
            time.sleep(1)
            continue
    # Tell the client process no more data will be forthcoming
    print("Time to kill some peons!")
    shared_job_q.put(POISONPILL)
    # Sleep a bit before shutting down the server - to give clients time to
    # realize the job queue is empty and exit in an orderly way.
    time.sleep(5)
    print("Aaaaaand we're done for the server!")
    manager.shutdown()
    print(results)

In [4]:
def make_client_manager(ip, port, authkey):
    """ Create a manager for a client. This manager connects to a server on the
        given address and exposes the get_job_q and get_result_q methods for
        accessing the shared queues from the server.
        Return a manager object.
    """
    class ServerQueueManager(BaseManager):
        pass

    ServerQueueManager.register('get_job_q')
    ServerQueueManager.register('get_result_q')

    manager = ServerQueueManager(address=(ip, port), authkey=authkey)
    manager.connect()

    print('Client connected to %s:%s' % (ip, port))
    return manager

# About the clients
## So we've got communications, what do we do with it?
* We've got Queue's to stuff full of work to do... but who's going to do _what_ ?
* Well... we can pass anything that _pickles_ (serializes) between processes
* Top-level functions and classes can be _pickled_ !
* So let's just pass the function that needs to be applied right along with the data!

In [5]:
def capitalize(word):
    """Capitalizes the word you pass in and returns it"""
    return word.upper()

## OK, I've got a function, now glue it all together
* We need to start a number of clients running 

In [6]:
def runclient(num_processes):
    manager = make_client_manager(IP, PORTNUM, AUTHKEY)
    job_q = manager.get_job_q()
    result_q = manager.get_result_q()
    run_workers(job_q, result_q, num_processes)
    
def run_workers(job_q, result_q, num_processes):
    processes = []
    for p in range(num_processes):
        temP = mp.Process(target=peon, args=(job_q, result_q))
        processes.append(temP)
        temP.start()
    print("Started %s workers!" % len(processes))
    for temP in processes:
        temP.join()

def peon(job_q, result_q):
    my_name = mp.current_process().name
    while True:
        try:
            job = job_q.get_nowait()
            if job == POISONPILL:
                job_q.put(POISONPILL)
                print("Aaaaaaargh", my_name)
                return
            else:
                try:
                    result = job['fn'](job['arg'])
                    print("Peon %s Workwork on %s!" % (my_name, job['arg']))
                    result_q.put({'job': job, 'result' : result})
                except NameError:
                    print("Can't find yer fun Bob!")
                    result_q.put({'job': job, 'result' : ERROR})

        except queue.Empty:
            print("sleepytime for", my_name)
            time.sleep(1)

In [7]:
POISONPILL = "MEMENTOMORI"
ERROR = "DOH"
IP = ''
PORTNUM = 5381
AUTHKEY = b'whathasitgotinitspocketsesss?'
data = ["Always", "look", "on", "the", "bright", "side", "of", "life!"]

In [8]:
server = mp.Process(target=runserver, args=(capitalize, data))
server.start()
time.sleep(1)
client = mp.Process(target=runclient, args=(4,))
client.start()
server.join()
client.join()

Server started at port 5381
Sending data!
Client connected to :5381
Started 4 workers!
Peon Process-2:1 Workwork on Always!
Peon Process-2:1 Workwork on on!Peon Process-2:2 Workwork on look!
Peon Process-2:3 Workwork on bright!Peon Process-2:1 Workwork on side!


Peon Process-2:3 Workwork on of!
Peon Process-2:2 Workwork on life!!Peon Process-2:4 Workwork on the!
sleepytime forsleepytime for
 sleepytime forsleepytime for Process-2:4 Process-2:1
Process-2:2 
Process-2:3

Got result! {'job': {'fn': <function capitalize at 0x7f77f8033d30>, 'arg': 'Always'}, 'result': 'ALWAYS'}
Got result! {'job': {'fn': <function capitalize at 0x7f77f8033d30>, 'arg': 'on'}, 'result': 'ON'}
Got result! {'job': {'fn': <function capitalize at 0x7f77f8033d30>, 'arg': 'bright'}, 'result': 'BRIGHT'}
Got result! {'job': {'fn': <function capitalize at 0x7f77f8033d30>, 'arg': 'look'}, 'result': 'LOOK'}
Got result! {'job': {'fn': <function capitalize at 0x7f77f8033d30>, 'arg': 'side'}, 'result': 'SIDE'}
Got result!

# Opdracht 3
## You wanted more? You get more!
* Gegeven het skelet dat ik hier heb geschetst, pas je Opdracht 1 zo aan dat:
    * Je het FastQ file op meerdere computers verwerkt
        * Je moet je script met een "-c" of "-s" optie duidelijk maken of deze in "server" of "client" modus start
        * M.a.w. of je vanuit de main() "runserver" of "runclient" aanroept.
        * Clients start je met de hand op een aantal binXXX of nucXXX computers op het netwerk. (tmux!)
        * Met de "-h" optie geef je aan welke host de server draait.
        * NB in eerste instantie kun je dit dus op 1 lokale computer doen door "-h localhost" te specificeren.
    * Probeer nu, als je dat al niet deed, het file in de _workers_ te openen. Duizenden regels over een networked Queue sturen werkt niet!
* Zie voor gedetailleerde command line en deliverables de Assignment in MS Teams.

# Questions?